# Sugarcane Leaf Disease Classification

This notebook implements a lightweight CNN model to classify sugarcane leaf images into six categories: Healthy, Bacterial Blight, Mosaic, RedRot, Rust, Yellow.

For testing trained model using this notebook please proceed to
[Pre-Trained Model](#scrollTo=zvJ6NeV-I1D4)

## Import Libraries

In [ ]:
%pip install tensorflow
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix

## Dataset Download
### Enter kaggle username and api key

## Dataset Download
### Enter kaggle username and api key

In [23]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/akilesh253/sugarcane-plant-diseases-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: joshuab13
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/akilesh253/sugarcane-plant-diseases-dataset


100%|██████████| 1.00G/1.00G [00:08<00:00, 125MB/s]


## Data Loading and Preprocessing

In [24]:
# Set parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = '/content/sugarcane-plant-diseases-dataset/Sugarcane_leafs'

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Validation data generator
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training generator
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Validation generator
validation_generator = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Class labels
class_names = list(train_generator.class_indices.keys())
print("Classes:", class_names)

Found 15943 images belonging to 6 classes.
Found 3983 images belonging to 6 classes.
Classes: ['BacterialBlights', 'Healthy', 'Mosaic', 'RedRot', 'Rust', 'Yellow']


## Model Building

In [ ]:
# Load MobileNetV2 base model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add custom layers
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(len(class_names), activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

## Training

In [ ]:
# Early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

## Evaluation

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.legend()

plt.show()

# Evaluate on validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_acc:.4f}')

# Predictions
validation_generator.reset()
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = validation_generator.classes

# Classification report
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.show()

### Model Conversion
Convert your trained Keras model into a TensorFlow Lite (TFLite) format.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('sugarcane_model.tflite', 'wb') as f:
    f.write(tflite_model)

## Testing Inference with TFLite

### Download Pre-trained Model
Download Pre-trained model for testing purposes without retraining.


In [25]:
import requests

url = "https://github.com/MoonLit-13/sugarcaneDiagnostic/raw/main/sugarcane_model.tflite"
model_filename = "sugarcane_model.tflite"

response = requests.get(url)
response.raise_for_status() # Raise an exception for HTTP errors

with open(model_filename, "wb") as f:
    f.write(response.content)

print(f"Successfully downloaded {model_filename}")

Successfully downloaded sugarcane_model.tflite


### Testing Inference with TFLite
- [Test Inference with multiple images](#scrollTo=SANvPevRQ9Zo)
- [Test Inference by uploading image](#scrollTo=1bf07004)

#### Testing Inference multiple with multiple images

### Download test images

In [30]:
import requests
import zipfile
import os

# URL to download the entire repository as a zip
repo_zip_url = "https://github.com/MoonLit-13/sugarcaneDiagnostic/archive/refs/heads/main.zip"
local_zip_filename = "sugarcaneDiagnostic-main.zip"
extracted_repo_folder = "sugarcaneDiagnostic-main" # Name of the folder after unzipping
target_images_dir = "/content/images"

print(f"Downloading {repo_zip_url}...")
try:
    response = requests.get(repo_zip_url, stream=True)
    response.raise_for_status() # Raise an exception for HTTP errors

    with open(local_zip_filename, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Successfully downloaded {local_zip_filename}.")

    # Unzip the repository
    print(f"Extracting {local_zip_filename}...")
    with zipfile.ZipFile(local_zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    print(f"Successfully extracted {local_zip_filename}.")

    # Define source and destination paths for test images
    source_test_images_path = os.path.join(extracted_repo_folder, "test Images")

    # Create target directory if it doesn't exist
    os.makedirs(target_images_dir, exist_ok=True)

    # Move contents from source_test_images_path to target_images_dir
    if os.path.exists(source_test_images_path):
        print(f"Moving contents of '{source_test_images_path}' to '{target_images_dir}'...")
        for item in os.listdir(source_test_images_path):
            s = os.path.join(source_test_images_path, item)
            d = os.path.join(target_images_dir, item)
            if os.path.isdir(s):
                import shutil
                shutil.move(s, d)
            else:
                os.rename(s, d)
        print("Test images moved successfully.")
    else:
        print(f"Warning: '{source_test_images_path}' not found in the extracted repository.")

    # Clean up downloaded zip and extracted repository folder
    if os.path.exists(local_zip_filename):
        os.remove(local_zip_filename)
        print(f"Removed {local_zip_filename}.")
    if os.path.exists(extracted_repo_folder):
        import shutil
        shutil.rmtree(extracted_repo_folder)
        print(f"Removed extracted repository folder {extracted_repo_folder}.")

    print("Test images download and setup complete.")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the repository: {e}")
except zipfile.BadZipFile as e:
    print(f"Error extracting the zip file (corrupted or not a zip): {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Successfully downloaded sugarcaneDiagnostic-main.zip.
Extracting sugarcaneDiagnostic-main.zip...
Successfully extracted sugarcaneDiagnostic-main.zip.
Moving contents of 'sugarcaneDiagnostic-main/test Images' to '/content/images'...
Test images moved successfully.
Removed sugarcaneDiagnostic-main.zip.
Removed extracted repository folder sugarcaneDiagnostic-main.
Test images download and setup complete.


In [31]:
# Test TFLite model with images in a directory (recursive)
from tensorflow import lite as tflite
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Class names (must match training order)
class_names = ['BacterialBlights', 'Healthy', 'Mosaic', 'RedRot', 'Rust', 'Yellow']

# Load TFLite model
interpreter = tflite.Interpreter(model_path='sugarcane_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Directory containing images (change this path as needed)
image_dir = "/content/images"   # Example path in Colab

# Walk through directory recursively
for root, dirs, files in os.walk(image_dir):
    for img_file_name in files:
        # Only process common image formats
        if img_file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            sample_img_path = os.path.join(root, img_file_name)

            # Load and preprocess image
            img = image.load_img(sample_img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0) / 255.0

            # Run inference
            interpreter.set_tensor(input_details[0]['index'], img_array.astype(np.float32))
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])

            # Get prediction
            predicted_index = np.argmax(output)
            predicted_class = class_names[predicted_index]
            confidence = output[0][predicted_index]

            print(f'Image: {sample_img_path}')
            print(f'Predicted Class: {predicted_class}')
            print(f'Confidence: {confidence:.4f}')
            print(f'All Probabilities: {output[0]}')
            print('-' * 50)

print("✅ Diagnostic process complete.")

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Image: /content/images/cropped_Bacterial Blight_510.png
Predicted Class: BacterialBlights
Confidence: 0.9925
All Probabilities: [9.9248499e-01 2.4496019e-06 4.8372176e-06 6.7616557e-03 2.5084818e-04
 4.9522740e-04]
--------------------------------------------------
Image: /content/images/cropped_yellow (280).jpeg
Predicted Class: Yellow
Confidence: 0.9209
All Probabilities: [2.5163191e-08 6.7935482e-02 3.2096400e-03 7.8321137e-03 1.2461223e-04
 9.2089820e-01]
--------------------------------------------------
Image: /content/images/cropped_yellow (276).jpeg
Predicted Class: Yellow
Confidence: 0.9314
All Probabilities: [1.5496171e-08 4.8475154e-02 3.4362609e-03 1.6379347e-02 3.5577972e-04
 9.3135339e-01]
--------------------------------------------------
Image: /content/images/cropped_Bacterial Blight_519.png
Predicted Class: BacterialBlights
Confidence: 0.9885
All Probabilities: [9.8851937e-01 1.7610854e-04 1.0511643e-03 3.0317453e-03 4.4789878e-03
 2.7426141e-03]
---------------------

#### Test inference with TFLite, upload an image when requested.

In [29]:
# Test TFLite model with uploaded images for diagnostic
from tensorflow import lite as tflite
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from google.colab import files # Import files for uploading

# Class names (must match training order)
class_names = ['BacterialBlights', 'Healthy', 'Mosaic', 'RedRot', 'Rust', 'Yellow']

# Load TFLite model
interpreter = tflite.Interpreter(model_path='sugarcane_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Please upload the images for diagnosis.")
uploaded = files.upload() # This will open a file upload dialog

if not uploaded:
    print("No images uploaded. Exiting.")
else:
    # Test each uploaded image
    for img_file_name, content in uploaded.items():
        # Save the uploaded file temporarily
        with open(img_file_name, 'wb') as f:
            f.write(content)

        sample_img_path = img_file_name

        # Load and preprocess image
        img = image.load_img(sample_img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], img_array.astype(np.float32))
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        # Get prediction
        predicted_index = np.argmax(output)
        predicted_class = class_names[predicted_index]
        confidence = output[0][predicted_index]

        print(f'Image: {img_file_name}')
        print(f'Predicted Class: {predicted_class}')
        print(f'Confidence: {confidence:.4f}')
        print(f'All Probabilities: {output[0]}')
        print('-' * 50)

        # Clean up the temporarily saved file
        os.remove(img_file_name)

print("Diagnostic process complete.")

Please upload the images for diagnosis.


Saving redrot image.jpg to redrot image.jpg
Image: redrot image.jpg
Predicted Class: RedRot
Confidence: 0.8218
All Probabilities: [1.3120713e-02 2.7136489e-06 3.5421390e-08 8.2182372e-01 1.0042624e-05
 1.6504274e-01]
--------------------------------------------------
Diagnostic process complete.
